__Задание__

В рамках этого итогового задания мы будем прогнозировать сердечную недостаточность.

Плана по выполнению задания не будет. 
Но есть несколько требований: 

- оберните весь конвейер преобразований в Pipeline

- подберите оптимальный вариант прогнозной модели с помощью GridSearchCV

- примените обученный на тренировочных данных конвейер к тестовым данным, никак не предобрабатывая их, а лишь загрузив из файла и отделив целевой признак от остальных

- получите на тестовой части качество не ниже 0.87 по метрике ROCAUC

Пояснение: если пропуски в новых данных и будут, то только в тех колонках, где они есть в тренировочной части.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import seaborn as sns
import pandas as pd
from scipy import stats
import plotly.express as px
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from summarytools import dfSummary

Загружаем данные обучающей выборки и проводим цикл анализа данных (EDA):

In [ ]:
train = pd.read_csv('heart_adapt_train.csv')

# Переименуем названия столбцов в соответствии с PEP8
train.columns = [re.sub('(?<!^)(?=[A-Z])|[ ]', '_', i, count=1).lower() for i in train.columns]
train.rename(columns={'s_t_slope': 'st_slope'}, inplace=True)
train.sample(3)

In [ ]:
# Возвращаем размерность
train.shape

- Данные представляют собой тренировочный набор информации о состоянии сердечно-сосудистой системы пациентов, который может быть использован для обучения моделей предсказания наличия ишемической болезни сердца.

- Таргет heart_disease представляет собой бинарную или двухклассовую переменную, где 0 означает отсутствие сердечной недостаточности, а 1 означает наличие сердечной недостаточности.

Таким образом, эта задача является задачей бинарной классификации.

DataFrame состоит из 12 столбцов, представляющих различные характеристики сердечно-сосудистого здоровья пациентов.

   - Age: возраст пациента в годах.
   - Sex: пол пациента (M - мужской, F - женский).
   - ChestPainType: тип болей в груди пациента (NAP - типичная стенокардия, ASY - безболевая ишемия, TA - атипичная стенокардия, ATA - атипичная стенокардия может подразумевать более выраженные или отличительные характеристики данного состояния).
   - RestingBP: артериальное давление пациента в состоянии покоя.
   - Cholesterol: уровень холестерина в крови пациента в данном случае уровень холестерина измеряется в миллиграммах на децилитр (мг/дл)
   - FastingBS: уровень глюкозы в крови пациента в состоянии голода (0 - норма, 1 - повышен).
   - RestingECG: результат электрокардиографического исследования в состоянии покоя (Normal - нормальный, ST - наличие изменений сегмента ST изменения сегмента ST могут указывать на ишемию миокарда, LVH - Гипертрофия (увеличение размера) левого желудочка сердца может быть вызвана различными заболеваниями, такими как артериальная гипертензия, пороки сердца, кардиомиопатии и другими).
   - MaxHR: максимальная частота сердечных сокращений пациента.
   - ExerciseAngina: наличие или отсутствие стенокардии при физической нагрузке (Y - да, N - нет).
   - Oldpeak: смещение сегмента ST во время нагрузочной пробы.
   - ST_Slope: наклон сегмента ST во время нагрузочной пробы (Up - вверх, Flat - плоский, Down - вниз).
   - HeartDisease: наличие или отсутствие ишемической болезни сердца (1 - есть, 0 - нет).

Таким образом, данные представляют собой информацию о состоянии сердечно-сосудистой системы пациентов, которая может быть использована для анализа и построения моделей предсказания наличия ишемической болезни сердца.

In [ ]:
# Применим библиотеку dfSummary к датафрейму обучающей выборки
dfSummary(train)

In [ ]:
display(train.loc[train['resting_bp'].isna()]) # выводим строки, в которых есть пропущенные значения в столбце resting_bp:

Заполним артериальное давление пациента медианой в соответствии с пояснением задания

In [ ]:
train.resting_bp = train.resting_bp.fillna(train.resting_bp.median()) 

In [ ]:
display(train.loc[train['age'].isna()]) # выводим строки, в которых есть пропущенные значения в столбце age:

In [ ]:
sns.histplot(train['age'], bins = 10)
plt.title('Распределение возраста пациентов');

In [ ]:
_, pvalue = stats.shapiro(train['age'])

alpha = .05

if pvalue < alpha:
    print(f'P-value:{pvalue:.5f}. Нулевая гипотеза отвергается')
else:
    print(f'P-value:{pvalue:.5f}. Подтверждается нулевая гипотеза')

In [ ]:
plt.figure(figsize=(14,6)) # Изменяем размер графика
sns.boxplot(train); # Смотрим графики распределения для обучающей выборки

Видим выбросы в столбце уровня холестерина в крови пациента, построим скетерплот для более детального анализа

In [ ]:
# Создание графика рассеяния с линией тренда OLS
fig = px.scatter(
    train, 
    x="age", 
    y="cholesterol", 
    trendline="ols",  # Добавление линии тренда OLS
    color="sex",  # Цвет для различия полов
    title="Взаимосвязь между возрастом и уровнем холестерина"
)

# Настройка подписей осей
fig.update_layout(
    xaxis_title="Возраст",
    yaxis_title="Уровень холестерина",
    legend_title="Пол"
)

# Добавление аннотации (пример)
highest_chol = train[train['cholesterol'] == train['cholesterol'].max()]
fig.add_annotation(
    x=highest_chol['age'].values[0],
    y=highest_chol['cholesterol'].values[0],
    text="Максимальный уровень холестерина",
    showarrow=True,
    arrowhead=2,
    ax=-40,  # Смещение по оси X
    ay=-30,  # Смещение по оси Y
    bgcolor="white",
    font=dict(size=14)
)

# Установка сетки для улучшения читаемости
fig.update_xaxes(showgrid=True)
fig.update_yaxes(showgrid=True)

# Отображение графика
fig.show()

In [ ]:
display(train.loc[train['cholesterol'] == 603].head()) # отображаем информацию про cholesterol со значением 603 мг/дл
display(train.loc[train['cholesterol'] == 529].head()) # отображаем информацию про cholesterol со значением 529 мг/дл
display(train.loc[train['cholesterol'] == 518].head()) # отображаем информацию про cholesterol со значением 518 мг/дл
display(train.loc[train['cholesterol'] == 491].head()) # отображаем информацию про cholesterol со значением 491 мг/дл

Значение 603.0, 529.0, 518.0, 491.0 для столбца "cholesterol" выглядят как выбросы. Стандартное отклонение составляет 58.6, а диапазон от 85.0 до 603.0 достаточно велик, что указывает на наличие аномальных значений. Удалим их из датафрейма.

In [ ]:
train.drop(train[train['cholesterol'] == 603].index, inplace=True) # удаляем cholesterol со значением 603 мг/дл
train.drop(train[train['cholesterol'] == 529].index, inplace=True) # удаляем cholesterol со значением 603 мг/дл
train.drop(train[train['cholesterol'] == 518].index, inplace=True) # удаляем cholesterol со значением 603 мг/дл
train.drop(train[train['cholesterol'] == 491].index, inplace=True) # удаляем cholesterol со значением 603 мг/дл

In [ ]:
# Создание графика рассеяния с линией тренда LOWESS
fig = px.scatter(
    train, 
    x="age", 
    y="max_hr", 
    trendline="lowess",  # Добавление линии тренда LOWESS
    color="sex",  # Использование цвета для различия полов
    title="Взаимосвязь между возрастом и максимальной частотой сердечных сокращений пациента"
)

# Настройка подписей осей
fig.update_layout(
    xaxis_title="Возраст",
    yaxis_title="Максимальная частота сердечных сокращений",
    legend_title="Пол"
)

# Установка сетки для улучшения читаемости
fig.update_xaxes(showgrid=True)
fig.update_yaxes(showgrid=True)

# Отображение графика
fig.show()

In [ ]:
# Создание графика рассеяния с регрессионной линией
fig = px.scatter(
    train, 
    x="age", 
    y="resting_bp", 
    color="sex",  # Используем цвет для различия пола
    title="Взаимосвязь между возрастом и артериальным давлением пациента в состоянии покоя",
    trendline="ols"  # Добавление линии тренда
)

# Настройка подписей осей
fig.update_layout(
    xaxis_title="Возраст",
    yaxis_title="Артериальное давление в состоянии покоя",
    legend_title="Пол"
)

# Установка сетки для улучшения читаемости
fig.update_xaxes(showgrid=True)
fig.update_yaxes(showgrid=True)

# Отображение графика
fig.show()

In [ ]:
# Создание гистограммы
fig = px.histogram(
    train, 
    x="age", 
    color="sex", 
    pattern_shape="sex", 
    title="Наличие ишемической болезни сердца у пациентов"
)

# Настройка подписей осей
fig.update_layout(
    xaxis_title="Возраст",
    yaxis_title="Количество случаев",
    legend_title="Пол"
)

# Добавление аннотации (пример, если вы знаете важные возрастные группы)
fig.add_annotation(
    x=58,
    y=50,  # само значение нужно будет скорректировать
    text="Высокий риск!",
    showarrow=True,
    arrowhead=2,
    ax=0,
    ay=-40,
    bgcolor="white",
    font=dict(size=14)
)

# Установка сетки
fig.update_xaxes(showgrid=True)
fig.update_yaxes(showgrid=True)

# Отображение графика
fig.show()

In [ ]:
train.shape

Загрузим тестовую выборку и переименуем названия столбцов в соответствии с PEP8

In [ ]:
test = pd.read_csv('heart_adapt_test.csv')

# Переименуем названия столбцов в соответствии с PEP8
test.columns = [re.sub('(?<!^)(?=[A-Z])|[ ]', '_', i, count=1).lower() for i in test]
test.rename(columns={'s_t_slope': 'st_slope'}, inplace=True)
test.sample(3)

In [ ]:
test.shape

In [ ]:
# Определяем размер обучающей и тестовой выборок
train_size = train.shape[0]  # Количество образцов в обучающей выборке
test_size = test.shape[0]     # Количество образцов в тестовой выборке
total_size = train_size + test_size  # Общий размер выборок

# Рассчитываем процентное соотношение
train_percentage = (train_size / total_size) * 100  # Процент обучающей выборки
test_percentage = (test_size / total_size) * 100     # Процент тестовой выборки

# Выводим результаты
print(f"Размер обучающей выборки: {train_size} ({train_percentage:.2f}%)")
print(f"Размер тестовой выборки: {test_size} ({test_percentage:.2f}%)")


Отделяем признаки от таргета.

In [ ]:
# Определение целевой переменной и признаков
train_target = train['heart_disease']
train_features = train.drop(['heart_disease'], axis=1)
test_target = test['heart_disease']
test_features = test.drop(['heart_disease'], axis=1)

In [ ]:
# Посмотрим на уникальные значения категориальных признаков
for column in train.select_dtypes(include=['object']).columns:
    print(f"{column}:", train[column].unique())

In [ ]:
# Определение категориальных и числовых столбцов
categorical_cols = train_features.select_dtypes(include=['object']).columns.tolist()
numerical_cols = train_features.select_dtypes(exclude=['object']).columns.tolist()

In [ ]:
# Создаем общий предобработчик с трансформерами для числовых и категориальных данных
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),  # Заполнение пропусков
            ('scaler', StandardScaler())  # Стандартизация числовых признаков
        ]), numerical_cols),
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),  # Заполнение пропусков для категориальных
            ('ohe', OneHotEncoder(handle_unknown='ignore'))  # Одноразовое кодирование категориальных признаков
        ]), categorical_cols)
    ]
)

In [ ]:
# Теперь создаем полный пайплайн с классификаторами
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classify', LogisticRegression(class_weight='balanced', random_state=0))
])

# Параметры для GridSearchCV
params = [
    {'classify': [LogisticRegression(class_weight='balanced', random_state=0)],
     'classify__C': [0.01, 0.1, 1, 10, 100]},
    {'classify': [DecisionTreeClassifier(class_weight='balanced', random_state=0)],
     'classify__max_depth': [2, 5, 10, 20]},
]

# Поиск лучших гиперпараметров на тренировочных данных
grid_search = GridSearchCV(pipeline, param_grid=params, cv=5, scoring='roc_auc')
grid_search.fit(X=train_features, y=train_target)

# Получаем информацию о наилучшей модели
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Получаем предсказания меток классов и вероятностей на тестовых данных
test_pred_proba = best_model.predict_proba(test_features)[:, 1]
test_pred = best_model.predict(test_features)

# Оценка качества модели на тестовой выборке
roc_auc = roc_auc_score(test_target, test_pred_proba)

# Выводим отчет о качестве классификации для лучшей модели
print("Лучшие параметры модели:")
if isinstance(best_model.named_steps['classify'], LogisticRegression):
    model_name = "LogisticRegression"
    model_params = f"Регуляризационный параметр модели: C = {best_params['classify__C']}"
elif isinstance(best_model.named_steps['classify'], DecisionTreeClassifier):
    model_name = "DecisionTreeClassifier"
    model_params = f"max_depth = {best_params['classify__max_depth']}"

print(f"Используемая модель: {model_name}")
print(model_params)
# Выводим отчет о качестве классификации
print("\nОтчет о качестве классификации:")
print(classification_report(test_target, test_pred))

# Вычисляем и выводим метрику ROC-AUC
print(f"\nROC-AUC Score: {roc_auc:.2f}")

### Вывод:

#### 1. Тип используемой модели:
   - Примененная модель: Логистическая регрессия. Это популярный метод для бинарной классификации, который показывает хорошие результаты при наличии линейной зависимости между признаками.

#### 2. Параметры модели:
   - Использованные параметры: 
     - class_weight=balanced: Данный параметр помогает сбалансировать классы в случае, если они не равномерны, что улучшает общую производительность модели при работе с несбалансированными данными.
     - random_state=0: Этот параметр обеспечивает воспроизводимость результатов, позволяя получать те же результаты при повторном обучении модели.

   - Регуляризационный параметр (C = 0.1): Низкое значение C указывает на более сильную регуляризацию, что может помочь избежать переобучения, особенно если обучающая выборка содержит шум или много признаков.

#### 3. Качество классификации:
   - Precision (точность):
     - Класс 0: 0.83
     - Класс 1: 0.91

     Высокий уровень точности для обоих классов указывает на то, что модель хорошо распознает положительные примеры (класс 1) и достаточно точно классифицирует отрицательные примеры (класс 0).

   - Recall (полнота):
     - Класс 0: 0.83
     - Класс 1: 0.91

     Значения полноты также высокие, что говорит о способности модели правильно находить объекты обоих классов.

   - F1-score:
     - Класс 0: 0.83
     - Класс 1: 0.91

     F1-score является сбалансированной мерой точности и полноты, и его высокие значения подтверждают общее хорошее качество классификации модели.

   - Accuracy (точность): Общая точность модели составляет 88%, что является впечатляющим показателем.

   - Средние показатели:
     - Macro average: 0.87 (средние значения без учета дисбаланса классов)
     - Weighted average: 0.88 (с учетом дисбаланса классов, взвешенный по количеству примеров в каждом классе)

#### 4. ROC-AUC Score:
   - Значение ROC-AUC составляет 0.91, что свидетельствует о том, что модель имеет отличные характеристики при различении классов. Это показатель указывает на высокую вероятность успешного различения положительных и отрицательных классов.

#### <span style="color: orange; font-weight: bold;">Заключение:</span>

В целом, модель логистической регрессии с указанными параметрами демонстрирует хорошие результаты по всем метрикам. Применение параметров, таких как class_weight=balanced, помогает справиться с проблемами несбалансированных данных, а регуляризация с параметром C=0.1 эффективно контролирует переобучение. 

Рекомендуется продолжить тестирование модели на новых данных и, возможно, рассмотреть другие алгоритмы или настройки, чтобы увидеть, можно ли достичь еще лучших результатов.